In [ ]:
#----------------------------------------------------------------
#sequence-to-sequence word-level chatbot (teacher forcing style).
#----------------------------------------------------------------

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("nrmal.csv")
questions = df["question"].astype(str).tolist()
answers = df["answer"].astype(str).tolist()

print(questions)
print(answers)


['hi', 'what is your name', 'what is your hobby', 'which is your favourite colour', 'your favourite food', 'your education', 'you know tamil', 'hi', 'what is your name', 'what is your hobby', 'which is your favourite colour', 'your favourite food', 'your education', 'you know tamil', 'your name', 'do you know tamil', 'my name is nirmal', 'my name is kumar', 'my name is star', 'my name is moon', 'my name is jhon what is your name']
['hello', 'chatbot', 'chating', 'blue', 'sweets', 'engineer', 'yes', 'hello', 'chatbot', 'chating', 'blue', 'sweets', 'engineer', 'yes', 'chatbot', 'yes i known tamil', 'Hi nirmal', 'Hi kumar', 'Hi star', 'Hi moonsun', 'chatbot']


In [27]:
from collections import Counter

def tokenize(sentences):
    words = []
    for s in sentences:
        words.extend(s.lower().split())
    return words

all_words = tokenize(questions + answers)
word_counts = Counter(all_words)

word2idx = {word: i+1 for i, (word, _) in enumerate(word_counts.items())}
idx2word = {i: w for w, i in word2idx.items()}

vocab_size = len(word2idx) + 1  # padding = 0
print("Vocab size:", vocab_size)
print(word2idx)


Vocab size: 32
{'hi': 1, 'what': 2, 'is': 3, 'your': 4, 'name': 5, 'hobby': 6, 'which': 7, 'favourite': 8, 'colour': 9, 'food': 10, 'education': 11, 'you': 12, 'know': 13, 'tamil': 14, 'do': 15, 'my': 16, 'nirmal': 17, 'kumar': 18, 'star': 19, 'moon': 20, 'jhon': 21, 'hello': 22, 'chatbot': 23, 'chating': 24, 'blue': 25, 'sweets': 26, 'engineer': 27, 'yes': 28, 'i': 29, 'known': 30, 'moonsun': 31}


In [28]:
def text_to_sequence(texts):
    return [[word2idx[w] for w in t.lower().split() if w in word2idx] for t in texts]

X_seq = text_to_sequence(questions)
y_seq = text_to_sequence(answers)

max_len = max(max(len(s) for s in X_seq), max(len(s) for s in y_seq))
print("Max length:", max_len)
print(X_seq)
print(y_seq)

Max length: 8
[[1], [2, 3, 4, 5], [2, 3, 4, 6], [7, 3, 4, 8, 9], [4, 8, 10], [4, 11], [12, 13, 14], [1], [2, 3, 4, 5], [2, 3, 4, 6], [7, 3, 4, 8, 9], [4, 8, 10], [4, 11], [12, 13, 14], [4, 5], [15, 12, 13, 14], [16, 5, 3, 17], [16, 5, 3, 18], [16, 5, 3, 19], [16, 5, 3, 20], [16, 5, 3, 21, 2, 3, 4, 5]]
[[22], [23], [24], [25], [26], [27], [28], [22], [23], [24], [25], [26], [27], [28], [23], [28, 29, 30, 14], [1, 17], [1, 18], [1, 19], [1, 31], [23]]


In [31]:
def pad_sequences(sequences, max_len):
    padded = np.zeros((len(sequences), max_len), dtype=np.int64)
    for i, seq in enumerate(sequences):
        padded[i, :len(seq)] = seq
    return padded

X = pad_sequences(X_seq, max_len)
y = pad_sequences(y_seq, max_len)

X = torch.tensor(X)
y = torch.tensor(y)
print(X)
print(y)

tensor([[ 1,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  3,  4,  5,  0,  0,  0,  0],
        [ 2,  3,  4,  6,  0,  0,  0,  0],
        [ 7,  3,  4,  8,  9,  0,  0,  0],
        [ 4,  8, 10,  0,  0,  0,  0,  0],
        [ 4, 11,  0,  0,  0,  0,  0,  0],
        [12, 13, 14,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  3,  4,  5,  0,  0,  0,  0],
        [ 2,  3,  4,  6,  0,  0,  0,  0],
        [ 7,  3,  4,  8,  9,  0,  0,  0],
        [ 4,  8, 10,  0,  0,  0,  0,  0],
        [ 4, 11,  0,  0,  0,  0,  0,  0],
        [12, 13, 14,  0,  0,  0,  0,  0],
        [ 4,  5,  0,  0,  0,  0,  0,  0],
        [15, 12, 13, 14,  0,  0,  0,  0],
        [16,  5,  3, 17,  0,  0,  0,  0],
        [16,  5,  3, 18,  0,  0,  0,  0],
        [16,  5,  3, 19,  0,  0,  0,  0],
        [16,  5,  3, 20,  0,  0,  0,  0],
        [16,  5,  3, 21,  2,  3,  4,  5]])
tensor([[22,  0,  0,  0,  0,  0,  0,  0],
        [23,  0,  0,  0,  0,  0,  0,  0],
        [24,  0,  0,  0,  0,  0, 

In [6]:
class ChatbotLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, embed_dim, batch_first=True)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        return self.fc(out)


In [7]:
embedding_dim = 128
model = ChatbotLSTM(vocab_size, embedding_dim)


In [10]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 200

for epoch in range(EPOCHS):
    model.train()

    outputs = model(X)
    loss = criterion(outputs.view(-1, vocab_size), y.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 0.5822
Epoch 20, Loss: 0.5766
Epoch 40, Loss: 0.5757
Epoch 60, Loss: 0.5753
Epoch 80, Loss: 0.5752
Epoch 100, Loss: 0.5751
Epoch 120, Loss: 0.5750
Epoch 140, Loss: 0.5750
Epoch 160, Loss: 0.5750
Epoch 180, Loss: 0.5749


In [12]:
import os

os.makedirs("models", exist_ok=True)

torch.save({
    "model_state": model.state_dict(),
    "word2idx": word2idx,
    "idx2word": idx2word,
    "max_len": max_len
}, "models/lstm_chatbot.pth")

print("Model saved successfully!")



Model saved successfully!


In [13]:
def predict_answer(question):
    model.eval()
    with torch.no_grad():
        seq = [word2idx.get(w, 0) for w in question.lower().split()]
        seq = seq[:max_len]
        padded = torch.zeros(1, max_len, dtype=torch.long)
        padded[0, :len(seq)] = torch.tensor(seq)

        output = model(padded)
        pred_ids = torch.argmax(output, dim=-1)[0]

        words = [idx2word[i.item()] for i in pred_ids if i.item() > 0]
        return " ".join(words)


In [14]:
test_question = "what is your name"
response = predict_answer(test_question)
print("Response:", response)


Response: chating chatbot chatbot kumar yes yes yes i


In [15]:
print("Chatbot is ready! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    response = predict_answer(user_input)
    print("Chatbot:", response)


Chatbot is ready! Type 'exit' to quit.
Chatbot: chating chatbot chatbot kumar yes yes yes i
Chatbot: yes i known tamil tamil tamil tamil i
Chatbot: Goodbye!


In [16]:
checkpoint = torch.load("models/lstm_chatbot.pth", map_location="cpu")

model.load_state_dict(checkpoint["model_state"])
word2idx = checkpoint["word2idx"]
idx2word = checkpoint["idx2word"]
max_len = checkpoint["max_len"]


In [17]:
path="models/lstm_chatbot.pth"
os.makedirs(os.path.dirname(path), exist_ok=True)


In [18]:
import torch
import torch.nn as nn


In [19]:
class ChatbotLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, embed_dim, batch_first=True)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        return self.fc(out)


In [20]:
path = "models/lstm_chatbot.pth"

checkpoint = torch.load(path, map_location="cpu")

word2idx = checkpoint["word2idx"]
idx2word = checkpoint["idx2word"]
max_len = checkpoint["max_len"]

vocab_size = len(word2idx) + 1
embedding_dim = 128   # MUST match training

model = ChatbotLSTM(vocab_size, embedding_dim)
model.load_state_dict(checkpoint["model_state"])
model.eval()

print("Model loaded successfully!")


Model loaded successfully!


In [21]:
def predict_answer(question):
    with torch.no_grad():
        tokens = question.lower().split()
        seq = [word2idx.get(w, 0) for w in tokens]

        seq = seq[:max_len]

        padded = torch.zeros(1, max_len, dtype=torch.long)
        padded[0, :len(seq)] = torch.tensor(seq)

        output = model(padded)

        pred_ids = torch.argmax(output, dim=-1)[0]

        words = [idx2word[i.item()] for i in pred_ids if i.item() > 0]
        return " ".join(words)


In [22]:
print(predict_answer("what is your name"))
print(predict_answer("how are you"))


chating chatbot chatbot kumar yes yes yes i
yes i yes i known tamil tamil tamil


In [23]:
print("Chatbot is ready! Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    response = predict_answer(user_input)
    print("Chatbot:", response)


Chatbot is ready! Type 'exit' to quit.
Chatbot: yes known tamil tamil tamil i i known
Chatbot: Goodbye!
